In [167]:
import requests
import json
import time as t
import pytz
from datetime import datetime, timedelta, date
import io
import os

In [168]:
wall_id_list_sc = ['-37160097','-46252034','-91933860','-127924894','-55155418', '-29559271'
                   '-83013121','-23637723','-89379348','-63066646','-84068967','-30698831','-36507793','-27737784']
wall_id_list_ufo = ['-18061985','-3403','-5783844','-42121567','-5798603','-60138573',
                    '-108756567','-51626109','-69519452','-170107','-38764280','-35353580','-78289865','-32108399']
wall_id_list_mys = ['-38984737','-51626109','-42981469','-78022224']

access_token = '16fed931172679afa79d5b507986f330942ae12524bd5bc415ca3114a25f2c58b78d0105a6a79f13ebb37'

#https://oauth.vk.com/authorize?client_id=6306832&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token

epoch_timestamp = 1356998400#GMT: Tuesday, 1 January 2013 00:00:00
API_URL = 'https://api.vk.com/method/'

In [169]:
def write_file(data,filename):
    with open(filename, mode='a') as file:
        json.dump(data,file,indent=2,ensure_ascii=False)

In [170]:
def get_posts(owner_id):
#     directory = folder_name+'/'+i
#     group_filename = ''+i+'.json'

#     if not os.path.exists(directory):
#         os.makedirs(directory)

#     group_file = open(directory+'/'+group_filename, 'w')        

    posts = []
    last_post_timestamp = 1

    while (last_post_timestamp<epoch_timestamp):

        response = requests.get('https://api.vk.com/method/execute.aa04201f?',params={'owner_id': owner_id, 'access_token': access_token}).json()

        print(response)
        posts.extend(response)
        last_post_timestamp = int(posts[-1:]['date'])
#         write_file(posts,directory+'/'+group_filename)

#     group_file.close()

In [171]:
def group_id_to_file():
    output_sci = []
    output_mys = []
    
    for i in wall_id_list_ufo:
        output_mys.append(i)
            
    for i in wall_id_list_mys:
        output_mys.append(i)
            
    for i in wall_id_list_sc:
        output_sci.append(i)
        
    with open('IDs_SCI.json','w') as file:
        json.dump(output_sci,file,indent=2,ensure_ascii=False)
        
    with open('IDs_MYS.json','w') as file:
        json.dump(output_mys,file,indent=2,ensure_ascii=False)

In [172]:
def recieve_posts(page_id, last_days):
    deadline = datetime(2018,1,1, tzinfo=pytz.utc) - timedelta(days=last_days)
    unix_stamp = int(deadline.strftime("%s"))
    print('Timestamp: '+str(unix_stamp))

    params = {'access_token': access_token,
              'id': page_id,
              'deadline':  unix_stamp if last_days != -1 else last_days
              }
    
    received_posts = []

    offset = 0
    ONGOING = True
    
    while ONGOING:
        params['offset'] = offset
        response = json.loads(requests.post(
            API_URL + 'execute.getPosts?', params=params).text)

        if not 'response' in response:
            print('Response Error!\n'+str(response))
            break

        received_data = response['response']
        
        for chunk in received_data:
            chunk_size = len(chunk['ids'])
            
            for i in range(chunk_size):
                post = dict()
                post['date'] = datetime.fromtimestamp(chunk['dates'][i])
                
                if  last_days == -1 or post['date'].year >= deadline.year and post['date'].month >= deadline.month and post['date'].day >= deadline.day:
                    post['id'] = chunk['ids'][i]
                    post['likes'] = chunk['likes'][i]
                    post['reposts'] = chunk['reposts'][i]
                    post['comments'] = chunk['comments'][i]
                    received_posts.append(post)
                    
            if 'stop' in chunk:
                ONGOING = False
                break
                
        offset += 1

    return received_posts

In [173]:
temp = recieve_posts(-42981469, 1825)

Timestamp: 1357081200


In [174]:
# for i in wall_id_list_mys:
#     temp = recieve_posts(i,1826)
#     write_file(temp, 'MYS_'+str(i))
    
# for i in wall_id_list_ufo:
#     temp = recieve_posts(i,1826)
#     write_file(temp, 'UFO_'+str(i))
    
# for i in wall_id_list_sc:
#     temp = recieve_posts(i,1826)
#     write_file(temp, 'SCI_'+str(i))
    